Author: Rudi Kreidenhuber, <Rudi.Kreidenhuber@gmail.com>, 
License: BSD (3-clause)

----

# Video EEG Monitoring Annotation visualizer

----


## Inputs:
 - .edf-files you wish to analyze go into ./data folder

## Run:
 - Press play :-)

## Outputs:
 - Found in results folder
 - Results for single files are put into a folder that matches the input-filename

----

## Howto:
 1. **Mark Events in EEG file using the following prefixes:**
 - e- --> EEG marker
 - s- --> Semiology marker
 - no prefix --> Everything else (clinical tests during/ after seizure)
 - i- --> Marker to ignore for focused analysis

 - One marker **must (!) contain "Beginn"** --> this is considered the seizure onset (if it is missing, onset is set to zero)
 - every marker **can** contain Beginn, for example:
 - Onset first seen in EEG --> Markername "e-asdBeginnfgh" --> would still be recognized as EEG marker and seizure onset
 2. **Save EEG file in .edf format and copy to ./data folder**
 - Every file in this folder is going to be analyzed, if it ends with .edf
 

In [1]:
# general imports
import os
from os.path import join
import glob
import mne
from mne import Report
import pandas as pd
import numpy as np
from utils import (get_parent_dir, extract_lab_sec, raw_to_df, extract_ordered_groups, save_plotly_to_html,
                        create_results_folders, plot_interactive_subplot_with_table,
                        plot_interactive_tables, plot_interactive_eeg_and_semio, plot_interactive_eventcount,
                        plot_interactive_testing_results, plot_interactive_EEG_results, plot_interactive_semio_results,
                        win_create_results_folders)

# plotly imports
import plotly as py
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
py.offline.init_notebook_mode(connected=True)

win = False

# grab .edfs
edfs = glob.glob("../data/*.edf")
if win:
    edfs = glob.glob("..\\data\\*.edf")

print("edfs found:\n", edfs)

if win:
    win_create_results_folders(edfs)
    subj_name =  os.getcwd().split("\\")[-2].split("VEEG_Event_Processor-")[-1]
else:
    create_results_folders(edfs)
    subj_name =  os.getcwd().split("/")[-2].split("VEEG_Event_Processor-")[-1]
print(subj_name)

edfs found:
 ['../data/SD_Anfall_u_Testung.edf', '../data/GM30091987_file1.edf', '../data/GM30091987_file3.edf', '../data/PC19012021_F2.edf', '../data/PC19012021_file3.edf', '../data/PC19012021_file1.edf', '../data/PC19012021_F1.edf', '../data/SE09031998_file2.edf', '../data/PC19012021_file2.edf', '../data/SE09031998_file1.edf', '../data/PC19012021_F3.edf', '../data/GM30091987_file2.edf']
VEEG_Event_Processor


----
## Save data
----

In [2]:
df = dict() 
e_events = dict()
s_events = dict()
t_events = dict()


for e in edfs:
    print(f"Now processing file: {e}")
    raw = mne.io.read_raw(e, preload=True)
    df[e], onset = raw_to_df(raw, e)

    e_events[e], s_events[e], t_events[e] = extract_ordered_groups(df[e]) 
    
    #save
    if win:
        csv_path = os.path.join("..", "results", e.split("\\")[-1].split(".")[0], "tables")
        e_file = e.split("\\")[-1].split(".")[0]
    else:
        csv_path = os.path.join("..", "results", e.split("/")[-1].split(".")[0], "tables")
        e_file = e.split("/")[-1].split(".")[0]
    tsv_name = "All_data_" + e_file + ".tsv"
    fname = os.path.join(csv_path, tsv_name)
    df[e].to_csv(fname, sep="\t")
    tsv_name = "EEG_data_" + e_file + ".tsv"
    fname = os.path.join(csv_path, tsv_name)
    e_events[e].to_csv(fname, sep="\t")
    tsv_name = "Semiology_data_" + e_file + ".tsv"
    fname = os.path.join(csv_path, tsv_name)
    s_events[e].to_csv(fname, sep="\t")
    tsv_name = "Testing_data_" + e_file + ".tsv"
    fname = os.path.join(csv_path, tsv_name)
    t_events[e].to_csv(fname, sep="\t")    

for idx, val in enumerate(df.keys()):
    if idx == 0:
        # all data vertical
        vconcat = df[val]
        # all data horizontal
        concat = df[val]
        source = "source_" + str(idx)
        concat[source] = val
        cols = list(concat)
        cols.insert(0, cols.pop(cols.index(source)))
        concat = concat.loc[:, cols]
        concat = concat.sort_values(by=["time_from_onset"])
        if "source" in concat.keys():
            concat.drop(columns=["source"], axis=1, inplace=True)
        concat["order_of_occurence"] = (1 + np.arange(len(concat.loc[:,"time_from_onset"])))
        # eeg, semio
        eeg_ga, semio_ga, test_ga = e_events[val], s_events[val], t_events[val]  # should be same keys as for e in edfs...

    if idx > 0:
        # all data vertical
        vnew_df = df[val]
        vconcat = pd.concat([vconcat, vnew_df], axis=0)
        # all data horizontal
        new_df = df[val]
        source = "source_" + str(idx)
        new_df[source] = val
        cols = list(new_df)
        cols.insert(0, cols.pop(cols.index(source)))
        new_df = new_df.loc[:, cols]
        if "source" in new_df.keys():
            new_df.drop(columns=["source"], axis=1, inplace=True)
        new_df["order_of_occurence"] = (1 + np.arange(len(new_df.loc[:,"time_from_onset"]))).astype(int)
        concat = pd.merge(concat, new_df, how="outer", on="description", suffixes=(" ", "  "))
        # eeg, semio
        ne, ns, nt = e_events[val], s_events[val], t_events[val]
        eeg_ga = pd.merge(eeg_ga, ne, how="outer", on="description", suffixes=(" ", "  ")) 
        semio_ga = pd.merge(semio_ga, ns, how="outer", on="description", suffixes=(" ", "  "))
        test_ga = pd.merge(test_ga, nt, how="outer", on="description", suffixes=(" ", "  "))

    idx += 1

if "source_0" in vconcat.keys():
    vconcat.drop(columns=["source_0"], axis=1, inplace=True)


# save grand averages
if win:
    eeg_ga.to_csv("..\\results\\grand_average\\tables\\EEG_data_grand_average.tsv", sep="\t")
    semio_ga.to_csv("..\\results\\grand_average\\tables\\Semiology_data_grand_average.tsv", sep="\t")
    test_ga.to_csv("..\\results\\grand_average\\tables\\Testing_data_grand_average.tsv", sep="\t")
    concat.to_csv("..\\results\\grand_average\\tables\\All_data_grand_average_horizontal.tsv", sep="\t")
    vconcat.to_csv("..\\results\\grand_average\\tables\\All_data_grand_average.tsv", sep="\t")

else:
    eeg_ga.to_csv("../results/grand_average/tables/EEG_data_grand_average.tsv", sep="\t")
    semio_ga.to_csv("../results/grand_average/tables/Semiology_data_grand_average.tsv", sep="\t")
    test_ga.to_csv("../results/grand_average/tables/Testing_data_grand_average.tsv", sep="\t")
    concat.to_csv("../results/grand_average/tables/All_data_grand_average_horizontal.tsv", sep="\t")
    vconcat.to_csv("../results/grand_average/tables/All_data_grand_average.tsv", sep="\t")

Now processing file: ../data/SD_Anfall_u_Testung.edf
Extracting EDF parameters from /home/idrael/DATA/git/VEEG_Event_Processor/data/SD_Anfall_u_Testung.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 311551  =      0.000 ...  1216.996 secs...
Error: No marker containing "Beginn" found, cannot determine seizure onset for file:  ../data/SD_Anfall_u_Testung.edf
Setting seizure onset to the beginning of the file
Now processing file: ../data/GM30091987_file1.edf
Extracting EDF parameters from /home/idrael/DATA/git/VEEG_Event_Processor/data/GM30091987_file1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 166911  =      0.000 ...   651.996 secs...
Now processing file: ../data/GM30091987_file3.edf
Extracting EDF parameters from /home/idrael/DATA/git/VEEG_Event_Processor/data/GM30091987_file3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Rea

In [13]:
e_events

{'../data/SD_Anfall_u_Testung.edf':                  source                      description  time_from_onset  \
 13  SD_Anfall_u_Testung        e-EEG Beginn bifrontale D           319.63   
 19  SD_Anfall_u_Testung                   e-rh Alpha 8/s           331.41   
 21  SD_Anfall_u_Testung      e-gen D + re fr-central shw           334.57   
 24  SD_Anfall_u_Testung  e-propagation li fronto central           343.88   
 27  SD_Anfall_u_Testung                       e-EEG-Ende           409.10   
 
     order_of_occurence  
 13                   1  
 19                   2  
 21                   3  
 24                   4  
 27                   5  ,
 '../data/GM30091987_file1.edf':               source              description  time_from_onset  \
 1   GM30091987_file1                 e-beginn             0.00   
 2   GM30091987_file1           e-Theta-T8-6hz             0.24   
 3   GM30091987_file1             e-thta-t8f10             2.50   
 4   GM30091987_file1       e-theta-te

In [59]:
# Create .xlsx-file
def write_excel_table(e_events=None, s_events=None):
    writer = pd.ExcelWriter ("Test_Excel_File.xlsx")
    
    # EEG-Events
    i = 1
    for e in e_events.keys():
        try:
            if e_events[e].empty:
                print(f"Empty EEG-List --> {e_events[e]}, omitting")
            else:
                df_e = pd.DataFrame(e_events[e], columns=["description"])
                _, file = os.path.split(e)
                df_e = df_e.rename(columns={"description": file.split(".edf")[0]})
                df_e.to_excel(writer, sheet_name="EEG", startcol=(i+1), startrow=3, header=True, index=False)
                #writer.save()
                i += 1
        except Exception as e:
            print(f"Excel-File: Something went wrong trying to parse EEG-Events for {e}")

    # Semiology-Events
    i = 1
    for s in s_events.keys():
        try:
            if s_events[s].empty:
                print(f"Empty Semiology-List --> {s_events[s]}, omitting")
            else:
                df_s = pd.DataFrame(s_events[s], columns=["description"])
                _, file = os.path.split(s)
                df_s = df_s.rename(columns={"description": file.split(".edf")[0]})
                df_s.to_excel(writer, sheet_name="Semiology", startcol=(i+1), startrow=3, header=True, index=False)
                writer.save()
                i += 1
        except Exception as e:
            print(f"Excel-File: Something went wrong trying to parse Semiology-Events for {s}")

    


write_excel_table(e_events, s_events)


# https://xlsxwriter.readthedocs.io/working_with_pandas.html

Empty EEG-List --> Empty DataFrame
Columns: [source, description, time_from_onset, order_of_occurence]
Index: [], omitting
Empty EEG-List --> Empty DataFrame
Columns: [source, description, time_from_onset, order_of_occurence]
Index: [], omitting
Empty EEG-List --> Empty DataFrame
Columns: [source, description, time_from_onset, order_of_occurence]
Index: [], omitting
Empty EEG-List --> Empty DataFrame
Columns: [source, description, time_from_onset, order_of_occurence]
Index: [], omitting
Empty EEG-List --> Empty DataFrame
Columns: [source, description, time_from_onset, order_of_occurence]
Index: [], omitting
Empty Semiology-List --> Empty DataFrame
Columns: [source, description, time_from_onset, order_of_occurence]
Index: [], omitting
Empty Semiology-List --> Empty DataFrame
Columns: [source, description, time_from_onset, order_of_occurence]
Index: [], omitting


----
# Interactive Visualization
----

In [ ]:
# Plots/report for single seizures
report_title = subj_name + " - Single seizure plots"
report = Report(subject=subj_name, title=report_title)

event_folders = glob.glob("../results/*")
if win:
    event_folders = glob.glob("..\\results\\*")
data = dict()
EEG = dict()
Semio = dict()
Test = dict()
interactive_plots = dict()

for e in event_folders:
    if win:
        source = e.split("\\")[-1].split(".")[0]
        sep = "\\"
    else:
        source = e.split("/")[-1].split(".")[0]
        sep = "/"
    tsv_path = join(e, "tables")
    
    tsv_name = "All_data_" + source + ".tsv"
    tsv = os.path.join(tsv_path, tsv_name)
    data[source] = pd.read_csv(tsv, sep="\t")
    tsv_name = "EEG_data_" + source + ".tsv"
    tsv = os.path.join(tsv_path, tsv_name)
    EEG[source] = pd.read_csv(tsv, sep="\t")    
    tsv_name = "Semiology_data_" + source + ".tsv"
    tsv = os.path.join(tsv_path, tsv_name)
    Semio[source] = pd.read_csv(tsv, sep="\t")
    tsv_name = "Testing_data_" + source + ".tsv"
    tsv = os.path.join(tsv_path, tsv_name)
    Test[source] = pd.read_csv(tsv, sep="\t")

    if source == "grand_average":
        pass
    else:
        interactive_plots[source] = plot_interactive_subplot_with_table(data[source], EEG[source], 
                                                                    Semio[source], Test[source], title=source)
        save_name = join("..", "results", source, "viz", str(source + "_interactive_viz.html"))
        if not os.path.isfile(save_name):
            save_plotly_to_html(interactive_plots[source], source=source)
            cap = source + " VIZ --> seizure"
            report.add_htmls_to_section(interactive_plots[source].to_html(full_html=False), 
                                        section=source, captions=cap)
        

        # event counts (plot.ly)
        event_counts = plot_interactive_eeg_and_semio(eeg=EEG[source], semio=Semio[source], source=source)
        cap = source + " VIZ --> event_conuts"
        sec = source
        report.add_htmls_to_section(event_counts.to_html(full_html=False), section=sec, captions=cap)

        # Testing
        cap = source + " VIZ --> Testing results"
        testing_viz = plot_interactive_testing_results(t_events=Test[source], title=cap)
        report.add_htmls_to_section(testing_viz.to_html(full_html=False), section=sec, captions=cap)

# Save all
report_save_name = "../results/Single_seizures_report.html"
if win:
    report_save_name = "..\\results\\Single_seizures_report.html"
report.save(report_save_name, overwrite=True)

In [ ]:
# Plots/report for grand average

ga_report_title = subj_name + " - All seizures"
ga_report = Report(subject=subj_name, title=ga_report_title)

source="grand_average"

EEG["grand_average"], Semio["grand_average"], Test["grand_average"] = extract_ordered_groups(df=data["grand_average"])

ga_fig = plot_interactive_subplot_with_table(df=data["grand_average"], eeg=EEG["grand_average"], 
                                                semio=Semio["grand_average"], testing=Test["grand_average"], title=ga_report_title)

save_name = join("..", "results", "grand_average", "viz", str("grand_average_interactive_viz.html"))
if not os.path.isfile(save_name):
    save_plotly_to_html(ga_fig, source=source)
    cap = source + " VIZ --> All seizures"
    ga_report.add_htmls_to_section(ga_fig.to_html(full_html=False), 
                                section=source, captions=cap)

# event counts (plot.ly)
event_counts = plot_interactive_eeg_and_semio(eeg=EEG[source], semio=Semio[source], source=source)
cap = source + " VIZ --> All event_conuts"
sec = source
ga_report.add_htmls_to_section(event_counts.to_html(full_html=False), section=sec, captions=cap)
# EEG
cap = source + " VIZ --> All EEG results"
eeg_viz = plot_interactive_EEG_results(e_events=EEG["grand_average"], title=cap)
ga_report.add_htmls_to_section(eeg_viz.to_html(full_html=False), section=sec, captions=cap)
# Semiology
cap = source + " VIZ --> All Testing results"
testing_viz = plot_interactive_testing_results(t_events=Test[source], title=cap)
ga_report.add_htmls_to_section(testing_viz.to_html(full_html=False), section=sec, captions=cap)
# Testing
cap = source + " VIZ --> All Semiology results"
semio_viz = plot_interactive_semio_results(s_events=Semio[source], title=cap)
ga_report.add_htmls_to_section(semio_viz.to_html(full_html=False), section=sec, captions=cap)

report_save_name = "../results/Grand_average_report.html"
if win:
    report_save_name = "..\\results\\Grand_average_report.html"
ga_report.save(report_save_name, overwrite=True)

## To do:

EKG?

Create a radar chart of EEG and Semiology signs (r/l hemisphere: front, temp, parietal, occipital) - https://plotly.com/python/radar-chart/

